## IMPORT

In [78]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [79]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [80]:
# cd /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0 

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
cd /content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/"model_v2.1"

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.1


In [83]:
print(os.getcwd())

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.1


## Preparing the dataset

In [84]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

In [85]:
import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

In [114]:
ratings = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    "user_id": x["user_id"],
    "add" : x["add"],
    "like" : x['like']
})
locations = wishEmbeddingDs.map(lambda x: x["location_name"])

In [115]:
for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'add': 0, 'like': 1, 'location_name': b'JAKARTA', 'user_id': b'93'}
{'add': 0, 'like': 1, 'location_name': b'DUBAI', 'user_id': b'112'}


In [116]:
tf.random.set_seed(42)

NUM_DATA = ratings.__len__().numpy()

shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [117]:
cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

## MODEL

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In [65]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add")

    user_embeddings, location_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 9s 75ms/step - root_mean_squared_error: 0.4576 - factorized_top_k/top_1_categorical_accuracy: 0.0016 - factorized_top_k/top_5_categorical_accuracy: 0.0365 - factorized_top_k/top_10_categorical_accuracy: 0.0771 - factorized_top_k/top_50_categorical_accuracy: 0.4031 - factorized_top_k/top_100_categorical_accuracy: 0.8123 - loss: 0.2091 - regularization_loss: 0.0000e+00 - total_loss: 0.2091
Epoch 2/3
43/43 [==============================] - 3s 74ms/step - root_mean_squared_error: 0.4470 - factorized_top_k/top_1_categorical_accuracy: 0.0041 - factorized_top_k/top_5_categorical_accuracy: 0.0391 - factorized_top_k/top_10_categorical_accuracy: 0.0807 - factorized_top_k/top_50_categorical_accuracy: 0.4077 - factorized_top_k/top_100_categorical_accuracy: 0.8194 - loss: 0.1997 - regularization_loss: 0.0000e+00 - total_loss: 0.1997
Epoch 3/3
22/22 [==============================] - 3s 101ms/step - root_mean_squared_error: 0.4425 - factorized_top_

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [90]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [91]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10
43/43 [==============================] - 5s 81ms/step - root_mean_squared_error: 0.5039 - factorized_top_k/top_1_categorical_accuracy: 0.0391 - factorized_top_k/top_5_categorical_accuracy: 0.1327 - factorized_top_k/top_10_categorical_accuracy: 0.2526 - factorized_top_k/top_50_categorical_accuracy: 0.6816 - factorized_top_k/top_100_categorical_accuracy: 0.9046 - loss: 3180.3630 - regularization_loss: 0.0000e+00 - total_loss: 3180.3630
Epoch 2/10
43/43 [==============================] - 4s 83ms/step - root_mean_squared_error: 0.5034 - factorized_top_k/top_1_categorical_accuracy: 0.2189 - factorized_top_k/top_5_categorical_accuracy: 0.4910 - factorized_top_k/top_10_categorical_accuracy: 0.6025 - factorized_top_k/top_50_categorical_accuracy: 0.9341 - factorized_top_k/top_100_categorical_accuracy: 0.9879 - loss: 3180.2772 - regularization_loss: 0.0000e+00 - total_loss: 3180.2772
Epoch 3/10
43/43 [==============================] - 3s 81ms/step - root_mean_squared_error: 0.5036 - f

In [68]:
model.evaluate(cached_test, return_dict=True)

22/22 [==============================] - 2s 71ms/step - root_mean_squared_error: 0.5110 - factorized_top_k/top_1_categorical_accuracy: 0.1007 - factorized_top_k/top_5_categorical_accuracy: 0.1453 - factorized_top_k/top_10_categorical_accuracy: 0.2354 - factorized_top_k/top_50_categorical_accuracy: 0.6379 - factorized_top_k/top_100_categorical_accuracy: 0.9454 - loss: 1348.8674 - regularization_loss: 0.0000e+00 - total_loss: 1348.8674


{'factorized_top_k/top_100_categorical_accuracy': 0.9453850388526917,
 'factorized_top_k/top_10_categorical_accuracy': 0.23539049923419952,
 'factorized_top_k/top_1_categorical_accuracy': 0.10067358613014221,
 'factorized_top_k/top_50_categorical_accuracy': 0.6379027962684631,
 'factorized_top_k/top_5_categorical_accuracy': 0.1452758014202118,
 'loss': 558.7587280273438,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.5109630823135376,
 'total_loss': 558.7587280273438}

In [69]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((locations.batch(100), locations.batch(100).map(model.location_model)))
)

# Get recommendations.
user_id = "45"
_, titles = index(tf.constant([user_id]))
print(f"Recommendations for Rolas : {titles[0, :5]}")

Recommendations for Rolas : [b'PRANCIS' b'MEDAN' b'TOKYO' b'BALI TOWER' b'GEOPARK CILETUH']


### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
43/43 [==============================] - 5s 79ms/step - root_mean_squared_error: 0.4559 - factorized_top_k/top_1_categorical_accuracy: 0.0342 - factorized_top_k/top_5_categorical_accuracy: 0.3442 - factorized_top_k/top_10_categorical_accuracy: 0.4356 - factorized_top_k/top_50_categorical_accuracy: 0.7003 - factorized_top_k/top_100_categorical_accuracy: 0.9091 - loss: 3180.5864 - regularization_loss: 0.0000e+00 - total_loss: 3180.5864
Epoch 2/3
43/43 [==============================] - 4s 92ms/step - root_mean_squared_error: 0.4473 - factorized_top_k/top_1_categorical_accuracy: 0.3804 - factorized_top_k/top_5_categorical_accuracy: 0.5874 - factorized_top_k/top_10_categorical_accuracy: 0.6358 - factorized_top_k/top_50_categorical_accuracy: 0.8447 - factorized_top_k/top_100_categorical_accuracy: 0.9649 - loss: 3180.5062 - regularization_loss: 0.0000e+00 - total_loss: 3180.5062
Epoch 3/3
22/22 [==============================] - 4s 96ms/step - root_mean_squared_error: 0.4431 - fact

## Model V2.1

In [120]:
class Model21(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation='relu')
      # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
      # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add")
    ratings = features.pop("like")

    user_embeddings, location_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Retrieval 

In [122]:
model = Model21(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [104]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10
43/43 [==============================] - 5s 81ms/step - root_mean_squared_error: 0.5039 - factorized_top_k/top_1_categorical_accuracy: 0.0391 - factorized_top_k/top_5_categorical_accuracy: 0.1327 - factorized_top_k/top_10_categorical_accuracy: 0.2526 - factorized_top_k/top_50_categorical_accuracy: 0.6816 - factorized_top_k/top_100_categorical_accuracy: 0.9046 - loss: 3180.3630 - regularization_loss: 0.0000e+00 - total_loss: 3180.3630
Epoch 2/10
43/43 [==============================] - 5s 125ms/step - root_mean_squared_error: 0.5034 - factorized_top_k/top_1_categorical_accuracy: 0.2189 - factorized_top_k/top_5_categorical_accuracy: 0.4910 - factorized_top_k/top_10_categorical_accuracy: 0.6025 - factorized_top_k/top_50_categorical_accuracy: 0.9341 - factorized_top_k/top_100_categorical_accuracy: 0.9879 - loss: 3180.2772 - regularization_loss: 0.0000e+00 - total_loss: 3180.2772
Epoch 3/10
43/43 [==============================] - 3s 80ms/step - root_mean_squared_error: 0.5036 - 

In [105]:
model.evaluate(cached_test, return_dict=True)

22/22 [==============================] - 2s 69ms/step - root_mean_squared_error: 0.5110 - factorized_top_k/top_1_categorical_accuracy: 0.1007 - factorized_top_k/top_5_categorical_accuracy: 0.1453 - factorized_top_k/top_10_categorical_accuracy: 0.2354 - factorized_top_k/top_50_categorical_accuracy: 0.6379 - factorized_top_k/top_100_categorical_accuracy: 0.9454 - loss: 1348.8674 - regularization_loss: 0.0000e+00 - total_loss: 1348.8674


{'factorized_top_k/top_100_categorical_accuracy': 0.9453850388526917,
 'factorized_top_k/top_10_categorical_accuracy': 0.23539049923419952,
 'factorized_top_k/top_1_categorical_accuracy': 0.10067358613014221,
 'factorized_top_k/top_50_categorical_accuracy': 0.6379027962684631,
 'factorized_top_k/top_5_categorical_accuracy': 0.1452758014202118,
 'loss': 558.7587280273438,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.5109630823135376,
 'total_loss': 558.7587280273438}

In [123]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10
43/43 [==============================] - 5s 79ms/step - root_mean_squared_error: 0.4977 - factorized_top_k/top_1_categorical_accuracy: 0.0391 - factorized_top_k/top_5_categorical_accuracy: 0.1327 - factorized_top_k/top_10_categorical_accuracy: 0.2526 - factorized_top_k/top_50_categorical_accuracy: 0.6816 - factorized_top_k/top_100_categorical_accuracy: 0.9046 - loss: 3180.3630 - regularization_loss: 0.0000e+00 - total_loss: 3180.3630
Epoch 2/10
43/43 [==============================] - 3s 79ms/step - root_mean_squared_error: 0.4979 - factorized_top_k/top_1_categorical_accuracy: 0.2189 - factorized_top_k/top_5_categorical_accuracy: 0.4910 - factorized_top_k/top_10_categorical_accuracy: 0.6025 - factorized_top_k/top_50_categorical_accuracy: 0.9341 - factorized_top_k/top_100_categorical_accuracy: 0.9879 - loss: 3180.2772 - regularization_loss: 0.0000e+00 - total_loss: 3180.2772
Epoch 3/10
43/43 [==============================] - 4s 83ms/step - root_mean_squared_error: 0.4978 - f

In [124]:
model.evaluate(cached_test, return_dict=True)

22/22 [==============================] - 2s 76ms/step - root_mean_squared_error: 0.4945 - factorized_top_k/top_1_categorical_accuracy: 0.1007 - factorized_top_k/top_5_categorical_accuracy: 0.1453 - factorized_top_k/top_10_categorical_accuracy: 0.2354 - factorized_top_k/top_50_categorical_accuracy: 0.6379 - factorized_top_k/top_100_categorical_accuracy: 0.9454 - loss: 1348.8674 - regularization_loss: 0.0000e+00 - total_loss: 1348.8674


{'factorized_top_k/top_100_categorical_accuracy': 0.9453850388526917,
 'factorized_top_k/top_10_categorical_accuracy': 0.23539049923419952,
 'factorized_top_k/top_1_categorical_accuracy': 0.10067358613014221,
 'factorized_top_k/top_50_categorical_accuracy': 0.6379027962684631,
 'factorized_top_k/top_5_categorical_accuracy': 0.1452758014202118,
 'loss': 558.7587280273438,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.49449408054351807,
 'total_loss': 558.7587280273438}

In [121]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((locations.batch(100), locations.batch(100).map(model.location_model)))
)

# Get recommendations.
user_id = "45"
_, titles = index(tf.constant([user_id]))
print(f"Recommendations for Rolas : {titles[0, :5]}")

Recommendations for Rolas : [b'PRANCIS' b'MEDAN' b'TOKYO' b'BALI TOWER' b'GEOPARK CILETUH']
